In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.connect('imdbCloned.db')
movies = pd.read_csv("csvfiles/movies.csv")
actors = pd.read_csv("csvfiles/actors.csv")
casting = pd.read_csv("csvfiles/casting.csv")
movies.to_sql('movies', conn, index=False)
actors.to_sql('actors', conn, index=False)
casting.to_sql('casting', conn, index=False)